In [3]:
from collections import deque
import numpy as np

def create_zip_chain(num_operators, num_xoperators, is_cx_first):
    """Create list 0,1,0,1,...
    If is_cx_first is True, then 1 is first, else 0 is first
    Args:
        n (_type_): _description_
        m (_type_): _description_
        is_cx_first (bool): _description_

    Returns:
        _type_: _description_
    """
    result = []
    while num_operators > 0 or num_xoperators > 0:
        if is_cx_first:
            if num_xoperators > 0:
                result.append(1)
                num_xoperators -= 1
            if num_operators > 0:
                result.append(0)
                num_operators -= 1
        else:   
            if num_operators > 0:
                result.append(0)
                num_operators -= 1
            if num_xoperators > 0:
                result.append(1)
                num_xoperators -= 1
    return result


class Instructor:
    def __init__(self, num_qubits):
        self.operators = []
        self.operator = []
        self.operator_temp = []
        self.xoperator = []
        self.xoperators = []
        self.instructors = deque()  # Thay list bằng deque
        self.num_qubits = num_qubits
        self.is_cx_first = False
        self.orders = []
        self.LUT = None

    def append(self, gate, index, param=0):
        self.instructors.append((gate, index, param))

    def operatoring(self):
        if self.instructors and self.instructors[0][0] == "cx":
            self.is_cx_first = True
        
        involved_qubits = set()  # Thay self.barriers bằng set
        while len(self.instructors) > 0:
            gate, index, _ = self.instructors[0]
            is_break = False
            
            if gate == "cx":
                involved_qubits.add(index[0])
                involved_qubits.add(index[1])
                self.xoperator.append(self.instructors.popleft()[1])  # Lấy index trực tiếp
                if (len(involved_qubits) == self.num_qubits and 
                    len(self.instructors) > 0 and 
                    self.instructors[0][0] != "cx"):
                    is_break = True
            else:
                if index not in involved_qubits:  # Kiểm tra bằng set
                    self.operator.append(self.instructors.popleft())
                else:
                    self.operator_temp.append(self.instructors.popleft())
            
            if is_break:
                if len(self.operator) > 0:
                    self.operators.append(self.operator)
                if len(self.xoperator) > 0:
                    self.xoperators.append(self.xoperator)
                self.instructors.extendleft(self.operator_temp[::-1])  # Thêm vào đầu hiệu quả
                self.operator = []
                self.operator_temp = []
                self.xoperator = []
                involved_qubits = set()  # Đặt lại set
        
        # Xử lý các phần còn lại
        if len(self.operator) > 0:
            self.operators.append(self.operator)
        if len(self.operator_temp) > 0:
            self.operators.append(self.operator_temp)
        if len(self.xoperator) > 0:
            self.xoperators.append(self.xoperator)
        
        self.orders = create_zip_chain(len(self.operators), len(self.xoperators), self.is_cx_first)
        return
    


In [4]:

%load_ext autoreload
%autoreload 2
ins = Instructor(4)
for i in range(10000):
	ins.append("h", 0)
	ins.append("rx", 1, 0.78)
	ins.append("h", 2)
	ins.append("h", 0)
	ins.append("cx", [0, 1])
	ins.append("h", 2)
	ins.append("h", 2)
	ins.append("ry", 0, 0.56)
	ins.append("cx", [1, 2])
	ins.append("h", 1)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 0)
	ins.append("h", 1)
	ins.append("h", 2)
	ins.append("h", 3)
	ins.append("h", 0)
	ins.append("h", 2)
	ins.append("cx", [1, 3])
ins.operatoring()
